# 🎲 GNU Backgammon Research Environment - Google Colab Demo

This notebook sets up the complete GNU Backgammon research environment in Google Colab using your project's infrastructure.

**✨ Features:**
- 🚀 Uses your project's Makefile for setup
- 📊 Interactive game execution
- 📁 Log file analysis
- 🔧 No local installation required!

**⚡ Quick Start:** Click "Run All" or run cells individually

## 📦 Step 1: Clone Project & Setup Environment

In [ ]:
import os
import subprocess
import sys
from pathlib import Path

# Clone the repository if not already present
if not Path('/content/gnubg-llm').exists():
    print("📥 Cloning GNU Backgammon Research Environment...")
    !git clone https://github.com/GodErlich/gnubg-llm.git /content/gnubg-llm
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already present")

# Change to project directory
os.chdir('/content/gnubg-llm')
print(f"📂 Working directory: {os.getcwd()}")

# Show project structure
print("\n🏗️ Project Structure:")
!ls -la

## 🔧 Step 2: Run Project Setup Using Makefile

In [ ]:
# Use the project's Makefile for proper setup
print("🎯 Running project setup using Makefile...")
print("This uses your project's infrastructure: make all")
print("=" * 60)

# Run make all (this will install gnubg and Python dependencies)
os.environ['PATH'] = '/usr/games:' + os.environ['PATH']
result = subprocess.run(['make', 'all'], capture_output=True, text=True)

print("📤 Setup Output:")
print(result.stdout)

if result.stderr:
    print("⚠️ Setup Warnings/Errors:")
    print(result.stderr)

if result.returncode == 0:
    print("\n✅ Project setup completed successfully using your Makefile!")
else:
    print(f"\n❌ Setup failed with return code: {result.returncode}")
    # Try individual steps if make all fails
    print("\n🔄 Trying individual setup steps...")
    !make deps
    !make install-gnubg
    !make install-python

## 🔧 Step 3: Install dependencies globally

In [ ]:
"""
Install globally for collab
"""

import subprocess
import os
import sys

def run_command(cmd, description):
    """Run a command and return success status"""
    print(f"🔧 {description}")
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"   ✅ Success")
            return True
        else:
            print(f"   ⚠️ Warning (exit code {result.returncode})")
            return False
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return False

def install_packages_for_gnubg():
    """Install packages using the proven multi-location method"""
    
    print("🎯 GNU Backgammon Package Installation")
    print("Using the proven method that works in Colab")
    print("=" * 50)
    
    packages = ["requests", "python-dotenv", "certifi", "charset-normalizer", "idna", "urllib3"]
    
    # Method 1: Standard global installation
    print("\n1️⃣ Global installation...")
    for pkg in packages:
        run_command(f"sudo pip3 install --upgrade {pkg}", f"Installing {pkg} globally")
    
    # Method 2: Install to system site-packages
    print("\n2️⃣ System site-packages installation...")
    python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
    system_site = f"/usr/lib/python{python_version}/site-packages"
    run_command(f"sudo pip3 install --target {system_site} --upgrade requests python-dotenv", 
                f"Installing to {system_site}")
    
    # Method 3: Install to multiple gnubg-compatible locations
    print("\n3️⃣ Multiple location installation...")
    locations = [
        "/usr/lib/python3/dist-packages",
        f"/usr/lib/python{python_version}/dist-packages",
        "/usr/local/lib/python3.10/site-packages"  # Common in many systems
    ]
    
    for location in locations:
        if os.path.exists(os.path.dirname(location)):
            run_command(f"sudo pip3 install --target {location} --upgrade requests python-dotenv",
                       f"Installing to {location}")
    
    # Method 4: Force reinstall
    print("\n4️⃣ Force reinstall...")
    run_command("sudo pip3 install --force-reinstall --no-deps requests", "Force reinstall requests")
    run_command("sudo pip3 install --force-reinstall --no-deps python-dotenv", "Force reinstall python-dotenv")

def test_gnubg_access():
    """Test if gnubg can access the packages"""
    
    print("\n🧪 Testing GNU Backgammon Package Access")
    print("=" * 45)
    
    tests = {
        "requests": "import requests; print('✅ requests works')",
        "dotenv": "import dotenv; print('✅ dotenv works')"
    }
    
    all_working = True
    
    for name, script in tests.items():
        test_file = f"test_{name}.py"
        with open(test_file, "w") as f:
            f.write(script)
        
        print(f"\n🔍 Testing {name}:")
        try:
            result = subprocess.run(f"gnubg -t -p {test_file}", shell=True, 
                                  capture_output=True, text=True, timeout=10)
            
            if result.returncode == 0 and "✅" in result.stdout:
                print(f"   ✅ {name}: SUCCESS")
            else:
                print(f"   ❌ {name}: FAILED")
                all_working = False
        except Exception as e:
            print(f"   ❌ {name}: ERROR - {e}")
            all_working = False
        
        # Clean up
        if os.path.exists(test_file):
            os.remove(test_file)
    
    return all_working

def create_app_wrapper():
    """Create app.py wrapper with path fixes"""
    
    if not os.path.exists("app.py"):
        print("\n⚠️ app.py not found, skipping wrapper creation")
        return False
    
    print("\n🔧 Creating app.py wrapper...")
    
    # Backup original
    if not os.path.exists("app.py.backup"):
        with open("app.py", "r") as f:
            original = f.read()
        with open("app.py.backup", "w") as f:
            f.write(original)
        print("   📋 Backup created: app.py.backup")
    
    # Create wrapper
    python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
    wrapper_code = f'''# GNU Backgammon Python package path fixes
import sys
import os

# Add multiple possible package locations to Python path
possible_paths = [
    "/usr/local/lib/python{python_version}/dist-packages",
    "/usr/lib/python3/dist-packages",
    "/usr/local/lib/python3/dist-packages", 
    "/usr/lib/python{python_version}/site-packages",
    "/usr/lib/python{python_version}/dist-packages",
    os.path.expanduser("~/.local/lib/python{python_version}/site-packages")
]

for path in possible_paths:
    if os.path.exists(path) and path not in sys.path:
        sys.path.insert(0, path)

'''
    
    with open("app.py.backup", "r") as f:
        original = f.read()
    
    with open("app.py", "w") as f:
        f.write(wrapper_code + "\n" + original)
    
    print("   ✅ app.py wrapper created")
    return True

def main():
    """Main function"""
    
    print("🚀 GNU Backgammon Package Fix")
    print("Based on the proven Colab solution")
    print("=" * 40)
    
    # Check if running as root for sudo operations
    if os.geteuid() != 0:
        print("ℹ️ This script requires sudo for package installation")
    
    # Step 1: Install packages
    install_packages_for_gnubg()
    
    # Step 2: Test access
    if test_gnubg_access():
        print("\n✅ Package installation SUCCESSFUL!")
        print("GNU Backgammon can now access Python packages")
    else:
        print("\n⚠️ Package installation had issues")
        print("Creating app.py wrapper as fallback...")
        create_app_wrapper()
    
    # Step 3: Final test
    print("\n🎮 Final test...")
    try:
        result = subprocess.run(["python3", "main.py", "--help"], 
                              capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print("✅ main.py script is accessible")
        else:
            print("⚠️ main.py script issues (check if file exists)")
    except Exception as e:
        print(f"⚠️ Could not test main.py: {e}")
    
    print(f"\n🎉 Setup complete!")
    print("Your GNU Backgammon environment should now work properly.")
    print("Run: python3 main.py --a1 RandomAgent --a2 BestMoveAgent --n 1")

if __name__ == "__main__":
    main()

## 🧪 Step 4: Verify Installation Using Project Scripts

In [ ]:
# Use the project's status check
print("🔍 Checking installation status using project's Makefile...")
!make status

print("\n🧪 Testing main.py script:")
result = subprocess.run(['python3', 'main.py', '--help'], capture_output=True, text=True)
print(result.stdout)

print("\n🎯 Environment ready for your research project!")

## 🎮 Step 5: Run Games Using Your Project Scripts

In [ ]:
# Function to run games using your main.py script
def run_research_game(agent1="RandomAgent", agent2="BestMoveAgent", num_games=1, debug=True):
    """Run games using your project's main.py script"""
    print(f"🎲 Running {num_games} game(s) using your project: {agent1} vs {agent2}")
    print("Using: python3 main.py with your agents")
    print("-" * 60)
    
    # Build command using your script
    cmd = ['python3', 'main.py', '--a1', agent1, '--a2', agent2, '--n', str(num_games)]
    if debug:
        cmd.append('--d')
    
    print(f"📝 Command: {' '.join(cmd)}")
    
    # Run your script
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    print("\n📊 Game Results:")
    print(result.stdout)
    
    if result.stderr:
        print("\n⚠️ Debug Info:")
        print(result.stderr)
    
    return result.returncode == 0

# Run a demo using your project's agents
print("🚀 Demo 1: RandomAgent vs BestMoveAgent (using your project infrastructure)")
success = run_research_game("RandomAgent", "BestMoveAgent", 1)
print(f"\n{'✅ Demo completed!' if success else '❌ Demo failed'}")

## 📁 Step 6: Analyze Generated Logs

In [ ]:
# Analyze logs generated by your project
print("📊 Analyzing logs generated by your project...")
print("=" * 50)

output_dir = Path('output')
if output_dir.exists():
    log_files = list(output_dir.glob('*.txt'))
    
    print(f"📁 Found {len(log_files)} log files from your games:")
    for log_file in log_files:
        size = log_file.stat().st_size
        print(f"  📄 {log_file.name} ({size} bytes)")
    
    if log_files:
        # Show content of most recent log
        latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
        print(f"\n📋 Latest game log ({latest_log.name}):")
        print("-" * 40)
        
        with open(latest_log, 'r') as f:
            content = f.read()
            if len(content) > 1500:
                print(content[:1500] + "\n\n... (truncated for display)")
            else:
                print(content)
                
        # Show file structure created by your project
        print("\n📂 Output directory structure:")
        !ls -la output/
else:
    print("📁 No output directory found. Run a game first!")
    print("Your project creates logs in the 'output' directory.")

## 🚀 Next Steps

Your GNU Backgammon research environment is now fully functional in Google Colab!

### 🎮 What You Can Do:

1. **Run More Games**: Use `!python3 main.py --a1 AGENT1 --a2 AGENT2 --n NUMBER`
2. **Analyze Logs**: Check the `output/` directory for detailed game logs
3. **Use Makefile**: All your Makefile commands work: `!make status`, `!make clean`, etc.
4. **Develop Agents**: Modify files in `src/agents/` to create custom agents
5. **Research**: Use all your project's capabilities for backgammon AI research

### 🔧 Your Project Features Working in Colab:
- ✅ **Makefile setup** - complete project installation
- ✅ **Agent system** - all your custom agents available
- ✅ **Logging system** - detailed game analysis
- ✅ **GNU Backgammon engine** - full functionality
- ✅ **Batch processing** - multiple game statistics

### 📚 Development:
- **Local Development**: Use GitHub Codespaces or Gitpod for full IDE experience
- **Team Collaboration**: Share this Colab notebook with team members
- **Research**: Export logs and results for further analysis

🎉 **Happy researching with your GNU Backgammon AI environment!**

In [ ]:
# Run any agent from the command line
!python main.py --a1 RandomAgent --a2 BestMoveAgent --n 1 --d